In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

# URL of APSRTC routes page
URL = "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile"

# Initialize driver
def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

# Load page
def load_page(driver, url):
    driver.get(url)
    time.sleep(5)

# Scrape bus routes (links + names)
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Scroll to end of bus list
def scroll_until_end(driver):
    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(3)
        new_page_source = driver.page_source
        if new_page_source == old_page_source:
            scrolling = False

# Scrape bus details for one route
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)

       
        try:
             wait = WebDriverWait(driver, 10)             
             apsrtc = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='APSRTC Buses']")))
             apsrtc.click()
             time.sleep(5)
        except:
            print(f"No View Buses button for {route_name}")

        scroll_until_end(driver)

        bus_cards= driver.find_elements(By.XPATH, "//li[contains(@class,'tupleWrapper')]")
        print(f"{len(bus_cards)} buses found for {route_name}")

        bus_details = []
        for bus in bus_cards:
            try:
                name = bus.find_element(By.XPATH, ".//div[contains(@class,'travelsName')]").text
                bus_type = bus.find_element(By.XPATH, ".//p[contains(@class,'busType')]").text
                rating = bus.find_element(By.XPATH, ".//div[contains(@class,'rating')]").text
                price = bus.find_element(By.XPATH, ".//p[contains(@class,'finalFare')]").text
                duration = bus.find_element(By.XPATH, ".//p[contains(@class,'duration')]").text

                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": name,
                    "Bus_Type": bus_type,
                    "Duration": duration,
                    "Rating": rating,
                    "Price": price
                }
                bus_details.append(bus_detail)
            except Exception as e:
                print(f"Error extracting bus in {route_name}: {e}")

        return bus_details

    except Exception as e:
        print(f"Error accessing {url}: {e}")
        return []


driver = initialize_driver()
load_page(driver, URL)

all_bus_details = []


try:
    all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
    for link, name in zip(all_bus_routes_link, all_bus_routes_name):
        print(f"Scraping route: {name}")
        route_bus_details = scrape_bus_details(driver, link, name)
        if route_bus_details:
            all_bus_details.extend(route_bus_details)

except Exception as e:
    print(f"Error in scraping routes: {e}")

# Convert to DataFrame and save
df = pd.DataFrame(all_bus_details)
df.to_csv('APSRTC_all_routes.csv', index=False)
print("Data saved successfully to apsrtc_all_routes_scraped.csv")

driver.quit()


In [2]:
import pandas as pd
df=pd.read_csv('D:\\Freelancing work\\Red-Bus-Project\Redbusproject\\apsrtc_all_routes_scraped.csv')

<>:2: SyntaxWarning: invalid escape sequence '\R'
<>:2: SyntaxWarning: invalid escape sequence '\R'
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_8276\1565672593.py:2: SyntaxWarning: invalid escape sequence '\R'
  df=pd.read_csv('D:\\Freelancing work\\Red-Bus-Project\Redbusproject\\apsrtc_all_routes_scraped.csv')


In [3]:
import re
import pandas as pd



# Function to extract rating and no_of_ratings with safe checks
def extract_rating_and_count(text):
    matches = re.findall(r'\d+(?:\.\d+)?', str(text))

    # Extract first as rating (float) if exists and not empty
    rating = float(matches[0]) if len(matches) >= 1 and matches[0] != '' else None
    
    # Extract second as no_of_ratings (int) if exists and not empty
    no_of_ratings = int(matches[1]) if len(matches) >= 2 and matches[1] != '' else None
    
    return pd.Series([rating, no_of_ratings])


df[['rating_value', 'No_of_Ratings']] = df['Rating'].apply(extract_rating_and_count)

# Define review_status based on rating_value
def status(rating):
    if rating is None:
        return 'Unknown'
    elif rating >= 4.5:
        return 'Excellent Service'
    elif rating >= 4:
        return 'Good Service'
    elif rating >=3:
        return 'Average'
    else:
        return 'Below Average'

df['Review_Status'] = df['rating_value'].apply(status)


df.drop(columns=['Rating'], inplace=True)


df.rename(columns={'rating_value': 'Rating'}, inplace=True)




In [4]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Duration,Price,Rating,No_of_Ratings,Review_Status
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 35188,VENNELA (A.C. SLEEPER),6h 5m,₹781,4.0,33.0,Good Service
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3563,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",6h 15m,₹436,3.6,29.0,Average
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 35266,VENNELA (A.C. SLEEPER),6h 5m,₹781,3.7,31.0,Average
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3590,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",6h 15m,₹436,3.5,22.0,Average
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3058,VENNELA (A.C. SLEEPER),6h 5m,₹781,3.9,43.0,Average
...,...,...,...,...,...,...,...,...,...
504,Madanapalli to Bangalore,https://www.redbus.in/bus-tickets/madanapalli-...,APSRTC - 6006,INDRA(A.C. Seater),2h 30m,₹279,3.5,97.0,Average
505,Madanapalli to Bangalore,https://www.redbus.in/bus-tickets/madanapalli-...,APSRTC - 5675,Express Non AC Seater 2+3,4h,₹173,3.5,50.0,Average
506,Madanapalli to Bangalore,https://www.redbus.in/bus-tickets/madanapalli-...,APSRTC - 7594,Express Non AC Seater 2+3,4h,₹173,4.0,21.0,Good Service
507,Madanapalli to Bangalore,https://www.redbus.in/bus-tickets/madanapalli-...,APSRTC - 5759,Express Non AC Seater 2+3,4h,₹173,3.6,78.0,Average


In [5]:
df.isnull().sum()


Route_Name        0
Route_Link        0
Bus_Name          0
Bus_Type          0
Duration          0
Price             0
Rating            0
No_of_Ratings    37
Review_Status     0
dtype: int64

In [6]:
df['No_of_Ratings'].fillna(0, inplace=True)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_8276\2493728059.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['No_of_Ratings'].fillna(0, inplace=True)


In [7]:
df.isnull().sum()

Route_Name       0
Route_Link       0
Bus_Name         0
Bus_Type         0
Duration         0
Price            0
Rating           0
No_of_Ratings    0
Review_Status    0
dtype: int64

In [8]:
df.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
504    False
505    False
506    False
507    False
508    False
Length: 509, dtype: bool

In [9]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Duration,Price,Rating,No_of_Ratings,Review_Status
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 35188,VENNELA (A.C. SLEEPER),6h 5m,₹781,4.0,33.0,Good Service
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3563,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",6h 15m,₹436,3.6,29.0,Average
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 35266,VENNELA (A.C. SLEEPER),6h 5m,₹781,3.7,31.0,Average
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3590,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",6h 15m,₹436,3.5,22.0,Average
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3058,VENNELA (A.C. SLEEPER),6h 5m,₹781,3.9,43.0,Average
...,...,...,...,...,...,...,...,...,...
504,Madanapalli to Bangalore,https://www.redbus.in/bus-tickets/madanapalli-...,APSRTC - 6006,INDRA(A.C. Seater),2h 30m,₹279,3.5,97.0,Average
505,Madanapalli to Bangalore,https://www.redbus.in/bus-tickets/madanapalli-...,APSRTC - 5675,Express Non AC Seater 2+3,4h,₹173,3.5,50.0,Average
506,Madanapalli to Bangalore,https://www.redbus.in/bus-tickets/madanapalli-...,APSRTC - 7594,Express Non AC Seater 2+3,4h,₹173,4.0,21.0,Good Service
507,Madanapalli to Bangalore,https://www.redbus.in/bus-tickets/madanapalli-...,APSRTC - 5759,Express Non AC Seater 2+3,4h,₹173,3.6,78.0,Average


In [12]:
import pymysql

In [17]:
!pip install psycopg2-binary


   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   --------- ---------------------

In [29]:
!pip install SQLAlchemy 


   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.1 MB 540.5 kB/s eta 0:00:03
   -------------- ------------------------- 0.8/2.1 MB 657.8 kB/s eta 0:00:03
   -------------- ------------------------- 0.8/2.1 MB 657.8 kB/s eta 0:00:03
   ------------------- -------------------- 1.0/2.1 MB 680.3 kB/s eta 0:00:02
   ------------------- -------------------- 1.0/2.1 MB 680.3 kB/s eta 0:00:02
   ------------------- -------------------- 1.0/2.1 MB 680.3 kB/s eta 0:00:02
   ------------------------ --------------- 1.3/2.1 MB 599.2 kB/s eta 0:00:02
   ------------------------ --------------- 1.3/2.1 MB 599.2 kB/s eta 0:00:02
   ----------------------------

In [35]:
import psycopg2

conn = psycopg2.connect(
        host="dpg-d1n3hkfdiees73emhn6g-a.oregon-postgres.render.com",
        database="db_apbus",
        user="db_apbus_user",
        password="ypJsjZYMOMqsy5wd2nX0Tm4WqWRuZj3t",
        port=5432
)
cursor = conn.cursor()



In [37]:
conn = psycopg2.connect(
            host='dpg-d1n3hkfdiees73emhn6g-a.oregon-postgres.render.com',
            database='db_apbus',
            user='db_apbus_user',
            password='ypJsjZYMOMqsy5wd2nX0Tm4WqWRuZj3t',
            port=5432
        )

In [38]:
mycursor=conn.cursor()

In [39]:
mycursor.execute("CREATE DATABASE AP_BUS_DETAILS")

ActiveSqlTransaction: CREATE DATABASE cannot run inside a transaction block


In [ ]:
mycursor.execute("SHOW DATABASES")
for x in mycursor:
    print(x)

In [ ]:
mycursor.execute("USE AP_BUS_DETAILS")  

In [41]:
mycursor.execute("""
CREATE TABLE AP (
    id SERIAL PRIMARY KEY,
    route_name VARCHAR(255),
    route_link VARCHAR(500),
    bus_name VARCHAR(255),
    bus_type VARCHAR(255),
    duration VARCHAR(100),
    price VARCHAR(50),
    rating FLOAT,
    no_of_ratings FLOAT,
    review_status VARCHAR(50)
)
""")

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [ ]:
insert_query = """
INSERT INTO AP_bus (
    route_name, route_link, bus_name, bus_type, duration,
    price, rating, no_of_ratings, review_status
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""


for index, row in df.iterrows():
    mycursor.execute(insert_query, (
        row['Route_Name'],
        row['Route_Link'],
        row['Bus_Name'],
        row['Bus_Type'],
        row['Duration'],
        row['Price'],
        row['Rating'],           
        row['No_of_Ratings'],
        row['Review_Status']
    ))

# Commit transaction
con.commit()

In [10]:
df.to_csv('output.csv', index=False)

In [11]:
print(df.columns)

Index(['Route_Name', 'Route_Link', 'Bus_Name', 'Bus_Type', 'Duration', 'Price',
       'Rating', 'No_of_Ratings', 'Review_Status'],
      dtype='object')
